# Create Athena Database Schema

In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Import PyAthena

In [9]:
from pyathena import connect

# Set Private bucket

In [13]:
s3_private_path = "s3://ads508-team4-raw"

# List files in the Private bucket

In [14]:
!aws s3 ls $s3_private_path

2022-03-19 18:39:22     691755 assets.csv
2022-03-19 18:39:29    4819081 demographics.csv
2022-03-19 18:39:36   14187619 plays.csv
2022-03-19 18:39:21   32759612 psychographics.csv
2022-03-19 18:39:42    1759323 users.csv


# Create Athena Database

In [4]:
database_name = "ads508team4-finalproject"